In [ ]:
import anndata
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc


#Load in the integrated spleen dataset
adata_pl_raw=anndata.read_h5ad('/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/spleen_merged_raw.h5ad')
adata_b_cells=anndata.read_h5ad("/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Final_Data/spleen_b_cells.h5ad")
adata_t_cells=anndata.read_h5ad("/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Final_Data/spleen_t_cells.h5ad")

In [ ]:
#GO Analysis
import wget
import os
from goatools import obo_parser
go_db_url=' http://purl.obolibrary.org/obo/go/go-basic.obo'
data_folder="/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Raw_data/data"# Replace this with your data folder

if(not os.path.isfile(data_folder+'/go-basic.obo')):
    go_obo = wget.download(go_db_url, data_folder+'/go-basic.obo')
else:
    go_obo = data_folder+'/go-basic.obo'

In [ ]:
#Create a id2gos format file using Trianotate
triannotate=pd.read_csv('/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Raw_data/aPlwal.pri.V2.genome.annots.tsv',sep='\t')
triannotate

In [ ]:
#Creating a GODAG for Pleurodeles

from collections import defaultdict

eggnog_mapper = defaultdict(list)
preferred_names = defaultdict(int)

for i, row in triannotate.iterrows():
    gene_id = row['#gene_id']
    preferred_name = row['EggNM.Preferred_name']
    go_terms = row['EggNM.GOs']

    if go_terms != '.':
        # Determine the key based on the preferred name and gene ID
        if preferred_name != '.':
            key = preferred_name
            # Update the preferred_names dictionary
            preferred_names[preferred_name] += 1
        else:
            key = gene_id
            # Check if the gene_id has appeared before
            if gene_id in preferred_names:
                # Use the preferred name from the first appearance
                key = f"{gene_id}-{preferred_names[gene_id]}"
                preferred_names[gene_id] += 1

        # Check if the key already exists in the dictionary
        if key in eggnog_mapper:
            # Append a suffix starting from -1
            suffix = 1
            new_key = f"{key}-{suffix}"
            while new_key in eggnog_mapper:
                suffix += 1
                new_key = f"{key}-{suffix}"
            key = new_key

        eggnog_mapper[key].append(go_terms)

# Convert defaultdict to a regular dictionary
eggnog_mapper = dict(eggnog_mapper)

#Remove values with only '.' in the values
eggnog_mapper_filtered = {}
gene_list = []
for key in eggnog_mapper.keys():
    if eggnog_mapper[key] == '.':
        continue
    else:
        eggnog_mapper_filtered[key] = eggnog_mapper[key]
        gene_list.append(key)

#Write the mapper to create a id2gos file
with open('/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/id2gos.txt', 'w') as f: #Change this to where you want the dataset
    for key, go_terms in eggnog_mapper.items():
        # Join the list of GO IDs with semi-colons
        go_ids_str = go_terms[0].replace(',', ';') if len(go_terms) == 1 else ';'.join(go_terms)
        # Write the gene name, a tab, and the list of GO IDs
        f.write(f"{key}\t{go_ids_str}\n")

In [ ]:
#Collecting the top 200 differentially expressed gene

#For T cells
sc.tl.rank_genes_groups(adata_t_cells,groupby='leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata_t_cells, n_genes=25, sharey=False)
cluster0=sc.get.rank_genes_groups_df(adata_t_cells,group='0',key='rank_genes_groups')
cluster1=sc.get.rank_genes_groups_df(adata_t_cells,group='1',key='rank_genes_groups')
cluster2=sc.get.rank_genes_groups_df(adata_t_cells,group='2',key='rank_genes_groups')
cluster3=sc.get.rank_genes_groups_df(adata_t_cells,group='3',key='rank_genes_groups')
cluster4=sc.get.rank_genes_groups_df(adata_t_cells,group='4',key='rank_genes_groups')
cluster5=sc.get.rank_genes_groups_df(adata_t_cells,group='5',key='rank_genes_groups')
cluster6=sc.get.rank_genes_groups_df(adata_t_cells,group='6',key='rank_genes_groups')
cluster7=sc.get.rank_genes_groups_df(adata_t_cells,group='7',key='rank_genes_groups')
cluster8=sc.get.rank_genes_groups_df(adata_t_cells,group='8',key='rank_genes_groups')
cluster0=cluster0['names'][:200]
cluster1=cluster1['names'][:200]
cluster2=cluster2['names'][:200]
cluster3=cluster3['names'][:200]
cluster4=cluster4['names'][:200]
cluster5=cluster5['names'][:200]
cluster6=cluster6['names'][:200]
cluster7=cluster7['names'][:200]
cluster8=cluster8['names'][:200]

#For B cells
sc.tl.rank_genes_groups(adata_b_cells,groupby='leiden',method='wilcoxon')
sc.pl.rank_genes_groups(adata_b_cells,n_genes=25,sharey=False)
cluster0_b=sc.get.rank_genes_groups_df(adata_b_cells,group='0',key='rank_genes_groups')
cluster1_b=sc.get.rank_genes_groups_df(adata_b_cells,group='1',key='rank_genes_groups')
cluster2_b=sc.get.rank_genes_groups_df(adata_b_cells,group='2',key='rank_genes_groups')
cluster3_b=sc.get.rank_genes_groups_df(adata_b_cells,group='3',key='rank_genes_groups')
cluster4_b=sc.get.rank_genes_groups_df(adata_b_cells,group='4',key='rank_genes_groups')
cluster5_b=sc.get.rank_genes_groups_df(adata_b_cells,group='5',key='rank_genes_groups')
cluster6_b=sc.get.rank_genes_groups_df(adata_b_cells,group='6',key='rank_genes_groups')
cluster7_b=sc.get.rank_genes_groups_df(adata_b_cells,group='7',key='rank_genes_groups')
cluster8_b=sc.get.rank_genes_groups_df(adata_b_cells,group='8',key='rank_genes_groups')
cluster9_b=sc.get.rank_genes_groups_df(adata_b_cells,group='9',key='rank_genes_groups')
cluster10_b=sc.get.rank_genes_groups_df(adata_b_cells,group='10',key='rank_genes_groups')
cluster11_b=sc.get.rank_genes_groups_df(adata_b_cells,group='11',key='rank_genes_groups')
cluster12_b=sc.get.rank_genes_groups_df(adata_b_cells,group='12',key='rank_genes_groups')
cluster0_b=cluster0_b['names'][:200]
cluster1_b=cluster1_b['names'][:200]
cluster2_b=cluster2_b['names'][:200]
cluster3_b=cluster3_b['names'][:200]
cluster4_b=cluster4_b['names'][:200]
cluster5_b=cluster5_b['names'][:200]
cluster6_b=cluster6_b['names'][:200]
cluster7_b=cluster7_b['names'][:200]
cluster8_b=cluster8_b['names'][:200]
cluster9_b=cluster9_b['names'][:200]
cluster10_b=cluster10_b['names'][:200]
cluster11_b=cluster11_b['names'][:200]
cluster12_b=cluster12_b['names'][:200]

In [ ]:
#Gene Ontology Enrichment Analysis

from goatools.go_enrichment import GOEnrichmentStudy
from goatools.anno.idtogos_reader import IdToGosReader
import scipy
import fisher
from goatools.base import download_go_basic_obo
def go_enrichment(cluster):
    
    id2gos_file_path = '/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Inter_data/id2gos.txt'

    # Path to the Gene Ontology OBO file
    obo_file='/home/nikvaku/snic2022-6-312/LabMemberScratchDir/Nikhilesh/Raw_data/data/go-basic.obo'
    obo_file =GODag(obo_file)
# Create an instance of IdToGosReader and read the gene to GO annotations
    id2gos_reader = IdToGosReader(id2gos_file_path)
    id2gos_dict = id2gos_reader.get_id2gos()

    # Background set of genes (all genes in your study)
    background_genes = set(adata_pl_raw.var_names)

    # Your set of genes of interest
    genes_of_interest = list(cluster)  # Replace with your actual gene IDs

    # Create an instance of GOEnrichmentStudy
    go_enrichment = GOEnrichmentStudy(
        background_genes, # List of all genes in my study
        id2gos_dict,# List of Pleuro genes
        obo_file,
        propagate_counts=False,# Set to True if you want to propagate counts up the GO hierarchy
        alpha=0.01,  # Set your desired significance level
        methods=['fdr_bh']  # Set the multiple testing correction method
    )

    # Run the GO enrichment analysis
    go_results = go_enrichment.run_study(genes_of_interest)
    return go_results

#Creating a filtered table for the GO analysis
def create_go_table(go_results):
    goea_results_sig = [r for r in go_results if r.p_fdr_bh < 0.05 and len(r.study_items) > 1]

    # Create DataFrame from the significant results
    GO = pd.DataFrame([{
        'GO': res.GO,
        'term': res.goterm.name,
        'class': res.goterm.namespace,
        'p': res.p_uncorrected,
        'p_corr': res.p_fdr_bh,
        'n_genes': res.ratio_in_study[0],
        'n_study': res.ratio_in_study[1],
        'n_go': len(res.goterm.get_all_parents()) + 1, # Assuming you want to count the term itself plus all parents
        'study_genes': list(res.study_items)
    } for res in goea_results_sig])
    
    return GO


#Plot the results of GOEA
def go_plot(dataframe):
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import pandas as pd
    
    df = dataframe.copy()
    # Convert p-values to a negative log scale to amplify differences
    df['-log10(p_corr)'] = -np.log10(df['p_corr'])

    # Sort the DataFrame based on the significance level of GO terms
    df_sorted = df.sort_values('-log10(p_corr)', ascending=False)

    # Select the top N significant terms for plotting
    N = 20
    df_top = df_sorted.head(N)
    
    # Normalize the number of genes for color mapping across the full range (blue to red)
    gene_counts_norm = (df_top['n_genes'] - df_top['n_genes'].min()) / (df_top['n_genes'].max() - df_top['n_genes'].min())
    colors = plt.cm.coolwarm(gene_counts_norm)

    # Create the bar plot
    plt.figure(figsize=(10, 8))
    ax = sns.barplot(x='-log10(p_corr)', y='term', data=df_top,
                     palette=colors,
                     edgecolor='black')

    # Create color bar for the number of genes
    sm = plt.cm.ScalarMappable(cmap="coolwarm", norm=plt.Normalize(vmin=df_top['n_genes'].min(), vmax=df_top['n_genes'].max()))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label('Number of Genes', rotation=270, labelpad=15)

    # Set the plot labels and title
    plt.xlabel('-log10(Adjusted P-Value)')
    plt.ylabel('GO Term')
    plt.title('Cluster12 Significant GO Terms by Adjusted P-Value')

    plt.show()

#Use these functions for the entire process.
go_results = go_enrichment(cluster12_b)
GO_table = create_go_table(go_results)
go_plot(GO_table)